**Imports**

In [1]:
from threading import Thread, Lock, current_thread # threading core
from queue import Queue # queue for safe data exchange

import time

## Queue Data Structure:

- Linear data structure.
- Thread Safe.
- Operates in FIFO (First In First Out) principle.
- Example: Customer waiting in queue (line) to be served, where a customer who came first is server first.
- Can be used as a task queue in multithreading or multiprocessing environment.
- [Docs](https://docs.python.org/3/library/queue.html)

**Operations In Queue:**

In [2]:
# Declare queue object
q = Queue()

**insert / put item in queue**

In [3]:
q.put(1) # 1
q.put(2) # 2, 1
q.put(3) # 3, 2, 1

**Get and remove first item.**

In [4]:
# Get and remove first item.
first = q.get()
print(first)

1


**Blocking untill items served:**
- ``Queue.join()``
- Similar to ``Thread.join()`` blocks the main thread.
- Blocks until all items in the queue have been gotten and processed.
- count of unfinished tasks goes up whenever an item is added **Queue.put(item)** to the queue.

**Completion of Task:**

- ``Queue.task_done()``
- Indicate that a formerly enqueued task is complete
- Used by consumer thread (or worker) to notify that the processing of task is completed.

> If a Queue.join() is blocking the main thread, it resumes when all items have been processed
i.e. for all the item in queue, **task_done()** was called.

## Example: Producer Consumer With Queue
- We put the task in the queue.
- More than one threads as a consumer (or worker).
- Consumer thread randomly picks the data and processed the item in queue.

![Thread Worker Task Queue](./ThreadTaskQueue.png)

**Define a task queue:**

In [5]:
task_queue = Queue()
lock = Lock()

**Worker method**
- Caution: Use a lock for critical section operation.

In [6]:
def worker(q, lock):
    while True:
        item = q.get() # if queue is empty, gets blocked and waits untill items available
        
        with lock:
            print(f'Task: [{item}] recieved by {current_thread()}')
            time.sleep(1) # wait for IO bound operation
            print(f'Finished processing [{item}]')
            print("\n")
        q.task_done()

**Consumer: Three worker threads:**
- Daemon Thread
> The threads which are always going to run in the background that provides supports to main or non-daemon threads, those background executing threads are considered as Daemon Threads. The Daemon Thread does not block the main thread from exiting and continues to run in the background.

In [7]:
for _ in range(0, 3):
    t = Thread(target=worker, args=(task_queue, lock))
    t.daemon = True
    t.start()

**Inserting 10 tasks in our task queue:**

In [8]:
for task in range(0, 10):
    task_queue.put(task)

Task: [0] recieved by <Thread(Thread-4, started daemon 140542480234240)>


**Wait untill all the items in queue served.**
- There's no need to block the threads we created.
- Main thread can be itself by calling task_queue.join()

In [9]:
task_queue.join()

Finished processing [0]


Task: [3] recieved by <Thread(Thread-4, started daemon 140542480234240)>
Finished processing [3]


Task: [4] recieved by <Thread(Thread-4, started daemon 140542480234240)>
Finished processing [4]


Task: [5] recieved by <Thread(Thread-4, started daemon 140542480234240)>
Finished processing [5]


Task: [6] recieved by <Thread(Thread-4, started daemon 140542480234240)>
Finished processing [6]


Task: [7] recieved by <Thread(Thread-4, started daemon 140542480234240)>
Finished processing [7]


Task: [2] recieved by <Thread(Thread-5, started daemon 140542000232192)>
Finished processing [2]


Task: [1] recieved by <Thread(Thread-6, started daemon 140541991839488)>
Finished processing [1]


Task: [8] recieved by <Thread(Thread-4, started daemon 140542480234240)>
Finished processing [8]


Task: [9] recieved by <Thread(Thread-5, started daemon 140542000232192)>
Finished processing [9]




**How does the threads die, even though in a worker we've infinite while loop?**
>Cause we've demonized the thread, i'e when the main thread stops its execution or is killed,
all the other threads are killed.

**Template: Alternative implementation without daemonizing the Threads**
- Signaling the breaking of infinite while loop.
- Worker below can be terminated by putting None in the task queue.

In [10]:
def worker(q, lock):
    while True:
        item = q.get() # if queue is empty, gets blocked and waits untill items available
        if item == None: # signaling the breaking of infinite loop
            break
        with lock:
            print(f'Task: [{item}] recieved by {current_thread()}')
            time.sleep(1) # wait for IO bound operation
            print(f'Finished processing [{item}]')
            print("\n")
        q.task_done()